# Goud Chain API Testing Suite

Comprehensive test notebook for all Goud Chain API endpoints with timing metrics and reporting.

**Features:**
- Create API keys and authenticate
- Bulk data submission (configurable batch size)
- Collection management and decryption
- Blockchain explorer and analytics
- Performance metrics with visualizations
- Final report generation

## 1. Configuration

Adjust these settings before running tests.

In [ ]:
# ============ Configuration ============

# API Endpoint (Docker internal network or external)
BASE_URL = "http://localhost:8080"  # Use this when running in Docker
# BASE_URL = "http://localhost:8080"  # Use this for external testing

# Test Parameters
BULK_SUBMISSION_COUNT = 100  # Number of blocks to create in bulk test
API_TIMEOUT = 30  # Request timeout in seconds
RETRY_ATTEMPTS = 3  # Number of retries for failed requests

# Debug Settings
VERBOSE_LOGGING = True  # Print detailed logs
SHOW_REQUEST_BODIES = False  # Print full request/response bodies

# Report Settings
EXPORT_TO_CSV = True  # Export results to CSV
EXPORT_TO_JSON = True  # Export results to JSON
GENERATE_CHARTS = True  # Generate timing charts
REPORT_OUTPUT_DIR = "../scratch/"  # Where to save reports

print("✅ Configuration loaded")
print(f"📡 API Endpoint: {BASE_URL}")
print(f"🔢 Bulk submission count: {BULK_SUBMISSION_COUNT}")

## 2. Setup & Utilities

Import libraries and define helper functions.

In [ ]:
import requests
import json
import time
import platform
import psutil
import socket
import sys
import traceback
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Global state storage
GLOBAL_STATE = {
    "api_key": None,
    "account_id": None,
    "session_token": None,
    "collection_ids": [],
    "test_results": [],
    "timing_data": {},
    "system_info": {},
    "server_health_history": [],
    "failed_dependencies": set(),
}

# Configure plotting style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

def capture_system_info() -> Dict[str, Any]:
    """Capture comprehensive system information"""
    try:
        cpu_count = psutil.cpu_count(logical=True)
        cpu_count_physical = psutil.cpu_count(logical=False)
        memory = psutil.virtual_memory()
        disk = psutil.disk_usage('/')
        
        system_info = {
            # System basics
            "hostname": socket.gethostname(),
            "platform": platform.platform(),
            "platform_system": platform.system(),
            "platform_release": platform.release(),
            "platform_version": platform.version(),
            "architecture": platform.machine(),
            "processor": platform.processor(),
            
            # CPU
            "cpu_count_logical": cpu_count,
            "cpu_count_physical": cpu_count_physical,
            "cpu_frequency_mhz": psutil.cpu_freq().current if psutil.cpu_freq() else "N/A",
            "cpu_percent": psutil.cpu_percent(interval=1),
            
            # Memory
            "memory_total_gb": round(memory.total / (1024**3), 2),
            "memory_available_gb": round(memory.available / (1024**3), 2),
            "memory_used_gb": round(memory.used / (1024**3), 2),
            "memory_percent": memory.percent,
            
            # Disk
            "disk_total_gb": round(disk.total / (1024**3), 2),
            "disk_used_gb": round(disk.used / (1024**3), 2),
            "disk_free_gb": round(disk.free / (1024**3), 2),
            "disk_percent": disk.percent,
            
            # Python environment
            "python_version": sys.version,
            "python_executable": sys.executable,
            
            # Network
            "ip_address": socket.gethostbyname(socket.gethostname()),
            
            # Timestamp
            "captured_at": datetime.now().isoformat(),
        }
        
        # Try to get Docker info if running in container
        try:
            with open('/proc/1/cgroup', 'r') as f:
                system_info["running_in_docker"] = 'docker' in f.read()
        except:
            system_info["running_in_docker"] = False
        
        return system_info
    
    except Exception as e:
        return {"error": f"Failed to capture system info: {str(e)}"}

# Capture system info on startup
GLOBAL_STATE["system_info"] = capture_system_info()

print("✅ Libraries imported")
print("✅ Global state initialized")
print("✅ System information captured")
print(f"\n📊 System Overview:")
print(f"   Platform: {GLOBAL_STATE['system_info'].get('platform_system')} {GLOBAL_STATE['system_info'].get('platform_release')}")
print(f"   CPU: {GLOBAL_STATE['system_info'].get('cpu_count_logical')} cores @ {GLOBAL_STATE['system_info'].get('cpu_percent')}% usage")
print(f"   Memory: {GLOBAL_STATE['system_info'].get('memory_available_gb')} GB available / {GLOBAL_STATE['system_info'].get('memory_total_gb')} GB total")
print(f"   Disk: {GLOBAL_STATE['system_info'].get('disk_free_gb')} GB free / {GLOBAL_STATE['system_info'].get('disk_total_gb')} GB total")
print(f"   Python: {sys.version.split()[0]}")
print(f"   Docker: {'Yes' if GLOBAL_STATE['system_info'].get('running_in_docker') else 'No'}")

In [ ]:
# ============ Helper Functions ============

class ErrorCategory:
    """Error type classification"""
    CONNECTION_ERROR = "connection_error"
    TIMEOUT = "timeout"
    HTTP_ERROR = "http_error"
    SERVER_CRASH = "server_crash"
    JSON_PARSE_ERROR = "json_parse_error"
    AUTHENTICATION_ERROR = "authentication_error"
    UNKNOWN = "unknown"

class ServerHealth:
    """Server health states"""
    ALIVE = "alive"
    DEGRADED = "degraded"
    CRASHED = "crashed"
    UNKNOWN = "unknown"

class APIClient:
    """Wrapper for Goud Chain API calls with timing, error handling, and retry logic"""
    
    def __init__(self, base_url: str, timeout: int = 30, retry_attempts: int = 3):
        self.base_url = base_url.rstrip('/')
        self.timeout = timeout
        self.retry_attempts = retry_attempts
        self.session = requests.Session()
    
    def _log(self, message: str, level: str = "INFO"):
        if VERBOSE_LOGGING or level == "ERROR":
            timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
            print(f"[{timestamp}] {level}: {message}")
    
    def _categorize_error(self, exception: Exception) -> str:
        """Categorize exception into error type"""
        if isinstance(exception, requests.exceptions.ConnectionError):
            return ErrorCategory.CONNECTION_ERROR
        elif isinstance(exception, requests.exceptions.Timeout):
            return ErrorCategory.TIMEOUT
        elif isinstance(exception, requests.exceptions.HTTPError):
            return ErrorCategory.HTTP_ERROR
        elif isinstance(exception, json.JSONDecodeError):
            return ErrorCategory.JSON_PARSE_ERROR
        else:
            return ErrorCategory.UNKNOWN
    
    def check_server_health(self) -> Tuple[str, Dict]:
        """Check if server is responsive. Returns (health_status, details)"""
        try:
            response = self.session.get(f"{self.base_url}/health", timeout=5)
            if response.status_code == 200:
                data = response.json()
                health_status = ServerHealth.ALIVE
                details = {
                    "status": "healthy",
                    "node_id": data.get("node_id", "unknown"),
                    "chain_length": data.get("chain_length", 0),
                    "peer_count": data.get("peer_count", 0),
                }
            else:
                health_status = ServerHealth.DEGRADED
                details = {
                    "status": "degraded",
                    "http_status": response.status_code,
                    "error": response.text[:200]
                }
        except requests.exceptions.ConnectionError:
            health_status = ServerHealth.CRASHED
            details = {"status": "crashed", "error": "Cannot connect to server"}
        except requests.exceptions.Timeout:
            health_status = ServerHealth.DEGRADED
            details = {"status": "degraded", "error": "Health check timeout"}
        except Exception as e:
            health_status = ServerHealth.UNKNOWN
            details = {"status": "unknown", "error": str(e)}
        
        # Record in history
        GLOBAL_STATE["server_health_history"].append({
            "timestamp": datetime.now().isoformat(),
            "health": health_status,
            "details": details
        })
        
        return health_status, details
    
    def request(self, method: str, endpoint: str, data: Optional[Dict] = None, 
                headers: Optional[Dict] = None, retry: bool = True) -> Tuple[Optional[Dict], float, bool, str, Optional[str]]:
        """Make HTTP request with timing and retry logic. 
        Returns (response_data, elapsed_time, success, error_category, stack_trace)"""
        url = f"{self.base_url}{endpoint}"
        headers = headers or {}
        
        if data:
            headers["Content-Type"] = "application/json"
        
        retry_count = 0
        max_retries = self.retry_attempts if retry else 1
        last_error = None
        last_error_category = None
        stack_trace = None
        
        while retry_count < max_retries:
            start_time = time.time()
            
            try:
                self._log(f"{method} {endpoint} (attempt {retry_count + 1}/{max_retries})")
                
                if method == "GET":
                    response = self.session.get(url, headers=headers, timeout=self.timeout)
                elif method == "POST":
                    response = self.session.post(url, json=data, headers=headers, timeout=self.timeout)
                else:
                    raise ValueError(f"Unsupported method: {method}")
                
                elapsed = time.time() - start_time
                
                if response.status_code >= 200 and response.status_code < 300:
                    try:
                        result = response.json() if response.content else {}
                    except json.JSONDecodeError as e:
                        self._log(f"❌ JSON parse error: {str(e)}", "ERROR")
                        return {"error": "Invalid JSON response", "raw": response.text[:500]}, elapsed, False, ErrorCategory.JSON_PARSE_ERROR, traceback.format_exc()
                    
                    self._log(f"✅ Success ({response.status_code}) in {elapsed:.3f}s")
                    return result, elapsed, True, None, None
                else:
                    self._log(f"❌ Failed ({response.status_code}): {response.text[:200]}", "ERROR")
                    error_category = ErrorCategory.HTTP_ERROR
                    if response.status_code == 401 or response.status_code == 403:
                        error_category = ErrorCategory.AUTHENTICATION_ERROR
                    return {"error": response.text, "status_code": response.status_code}, elapsed, False, error_category, None
            
            except requests.exceptions.ConnectionError as e:
                elapsed = time.time() - start_time
                last_error = e
                last_error_category = ErrorCategory.CONNECTION_ERROR
                stack_trace = traceback.format_exc()
                self._log(f"❌ Connection error: {str(e)}", "ERROR")
                
                # Check if server crashed
                health, _ = self.check_server_health()
                if health == ServerHealth.CRASHED:
                    last_error_category = ErrorCategory.SERVER_CRASH
                    return {"error": "Server crashed or unreachable"}, elapsed, False, ErrorCategory.SERVER_CRASH, stack_trace
            
            except requests.exceptions.Timeout as e:
                elapsed = time.time() - start_time
                last_error = e
                last_error_category = ErrorCategory.TIMEOUT
                stack_trace = traceback.format_exc()
                self._log(f"❌ Timeout after {elapsed:.3f}s", "ERROR")
            
            except Exception as e:
                elapsed = time.time() - start_time
                last_error = e
                last_error_category = self._categorize_error(e)
                stack_trace = traceback.format_exc()
                self._log(f"❌ Exception: {str(e)}", "ERROR")
            
            retry_count += 1
            
            # Exponential backoff before retry
            if retry_count < max_retries and retry:
                backoff = min(2 ** retry_count, 10)  # Max 10 seconds
                self._log(f"⏳ Retrying in {backoff}s...")
                time.sleep(backoff)
        
        # All retries failed
        return {
            "error": str(last_error) if last_error else "Unknown error",
            "retry_count": retry_count
        }, elapsed, False, last_error_category or ErrorCategory.UNKNOWN, stack_trace

def record_test_result(test_name: str, success: bool, elapsed: float, details: Dict = None, 
                      error_category: Optional[str] = None, stack_trace: Optional[str] = None,
                      skipped: bool = False, skip_reason: Optional[str] = None, retry_count: int = 0):
    """Record test result for final report with enhanced metadata"""
    
    # Get current server health
    server_health = GLOBAL_STATE["server_health_history"][-1]["health"] if GLOBAL_STATE["server_health_history"] else ServerHealth.UNKNOWN
    
    result = {
        "test_name": test_name,
        "success": success,
        "elapsed_seconds": elapsed,
        "timestamp": datetime.now().isoformat(),
        "details": details or {},
        "error_category": error_category,
        "stack_trace": stack_trace,
        "skipped": skipped,
        "skip_reason": skip_reason,
        "retry_count": retry_count,
        "server_health": server_health,
    }
    GLOBAL_STATE["test_results"].append(result)
    
    # Track timing by endpoint (only for successful tests)
    if success and not skipped:
        if test_name not in GLOBAL_STATE["timing_data"]:
            GLOBAL_STATE["timing_data"][test_name] = []
        GLOBAL_STATE["timing_data"][test_name].append(elapsed)
    
    # Track failed dependencies
    if not success and not skipped:
        GLOBAL_STATE["failed_dependencies"].add(test_name)

def check_dependencies(*required_tests) -> Tuple[bool, Optional[str]]:
    """Check if required tests have passed. Returns (dependencies_met, skip_reason)"""
    for test in required_tests:
        if test in GLOBAL_STATE["failed_dependencies"]:
            return False, f"Dependency failed: {test}"
    return True, None

def run_test_with_error_handling(test_name: str, test_func, dependencies: List[str] = None):
    """Universal test wrapper with error handling and dependency checking"""
    dependencies = dependencies or []
    
    # Check dependencies
    deps_met, skip_reason = check_dependencies(*dependencies)
    if not deps_met:
        print(f"\n⏭️  Test skipped: {skip_reason}")
        record_test_result(test_name, False, 0, skipped=True, skip_reason=skip_reason)
        return None
    
    # Check server health before test
    print(f"\n🏥 Checking server health...")
    health, health_details = api.check_server_health()
    print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
    
    if health == ServerHealth.CRASHED:
        print(f"\n💀 Server crashed! Cannot run test.")
        record_test_result(test_name, False, 0, 
                          details={"server_crashed": True}, 
                          error_category=ErrorCategory.SERVER_CRASH,
                          skipped=True,
                          skip_reason="Server crashed")
        return None
    
    # Run test
    try:
        start_time = time.time()
        result = test_func()
        elapsed = time.time() - start_time
        return result
    except Exception as e:
        elapsed = time.time() - start_time
        error_msg = str(e)
        stack = traceback.format_exc()
        
        print(f"\n💥 UNHANDLED EXCEPTION in test '{test_name}':")
        print(f"   Error: {error_msg}")
        if VERBOSE_LOGGING:
            print(f"   Stack trace:\n{stack}")
        
        record_test_result(test_name, False, elapsed,
                          details={"unhandled_exception": error_msg},
                          error_category=ErrorCategory.UNKNOWN,
                          stack_trace=stack)
        return None

def print_section(title: str):
    """Print formatted section header"""
    print(f"\n{'='*60}")
    print(f"  {title}")
    print(f"{'='*60}\n")

# Initialize API client
api = APIClient(BASE_URL, API_TIMEOUT, RETRY_ATTEMPTS)

print("✅ Helper functions defined")
print("✅ API client initialized with retry logic")
print(f"   Max retries: {RETRY_ATTEMPTS}")
print(f"   Timeout: {API_TIMEOUT}s")

## 3. Test: Create Account

Create a new user account and obtain an API key.

In [ ]:
print_section("Test 1: Create Account")

try:
    # Check server health
    print("🏥 Checking server health...")
    health, health_details = api.check_server_health()
    print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
    
    if health == ServerHealth.CRASHED:
        print("\n💀 Server is not responding! Cannot create account.")
        record_test_result("Create Account", False, 0, 
                          details={"server_crashed": True}, 
                          error_category=ErrorCategory.SERVER_CRASH,
                          skipped=True,
                          skip_reason="Server crashed")
    else:
        request_data = {
            "metadata": "Test account created from Jupyter notebook"
        }
        
        response, elapsed, success, error_category, stack_trace = api.request("POST", "/account/create", data=request_data)
        
        if success:
            GLOBAL_STATE["api_key"] = response.get("api_key")
            GLOBAL_STATE["account_id"] = response.get("account_id")
            
            print(f"\n✅ Account created successfully!")
            print(f"   Account ID: {GLOBAL_STATE['account_id']}")
            print(f"   API Key: {GLOBAL_STATE['api_key'][:20]}..." if GLOBAL_STATE['api_key'] else "   API Key: None")
            if response.get('warning'):
                print(f"\n⚠️  {response.get('warning')}")
        else:
            print(f"\n❌ Failed to create account: {response.get('error', 'Unknown error')}")
            print(f"   Error category: {error_category}")
        
        record_test_result("Create Account", success, elapsed, response, 
                          error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Create Account", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 4. Test: Login

Login with API key to obtain a session token.

In [ ]:
print_section("Test 2: Login")

try:
    # Check dependencies
    deps_met, skip_reason = check_dependencies("Create Account")
    if not deps_met:
        print(f"\n⏭️  Test skipped: {skip_reason}")
        record_test_result("Login", False, 0, skipped=True, skip_reason=skip_reason)
    elif not GLOBAL_STATE["api_key"]:
        print("❌ No API key available. Create Account test did not provide an API key.")
        record_test_result("Login", False, 0, skipped=True, skip_reason="No API key available")
    else:
        # Check server health
        print("🏥 Checking server health...")
        health, health_details = api.check_server_health()
        print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
        
        if health == ServerHealth.CRASHED:
            print("\n💀 Server is not responding! Cannot login.")
            record_test_result("Login", False, 0, 
                              details={"server_crashed": True}, 
                              error_category=ErrorCategory.SERVER_CRASH,
                              skipped=True,
                              skip_reason="Server crashed")
        else:
            request_data = {
                "api_key": GLOBAL_STATE["api_key"]
            }
            
            response, elapsed, success, error_category, stack_trace = api.request("POST", "/account/login", data=request_data)
            
            if success:
                GLOBAL_STATE["session_token"] = response.get("session_token")
                
                print(f"\n✅ Login successful!")
                print(f"   Session Token: {GLOBAL_STATE['session_token'][:30]}..." if GLOBAL_STATE['session_token'] else "   Session Token: None")
                print(f"   Expires in: {response.get('expires_in')} seconds")
            else:
                print(f"\n❌ Login failed: {response.get('error', 'Unknown error')}")
                print(f"   Error category: {error_category}")
            
            record_test_result("Login", success, elapsed, response,
                              error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Login", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 5. Test: Submit Data (Single)

Submit a single encrypted collection to the blockchain.

In [ ]:
print_section("Test 3: Submit Data (Single)")

try:
    # Check dependencies
    deps_met, skip_reason = check_dependencies("Create Account")
    if not deps_met:
        print(f"\n⏭️  Test skipped: {skip_reason}")
        record_test_result("Submit Data (Single)", False, 0, skipped=True, skip_reason=skip_reason)
    elif not GLOBAL_STATE["api_key"]:
        print("❌ No API key available. Create Account test did not provide an API key.")
        record_test_result("Submit Data (Single)", False, 0, skipped=True, skip_reason="No API key available")
    else:
        # Check server health
        print("🏥 Checking server health...")
        health, health_details = api.check_server_health()
        print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
        
        if health == ServerHealth.CRASHED:
            print("\n💀 Server is not responding! Cannot submit data.")
            record_test_result("Submit Data (Single)", False, 0, 
                              details={"server_crashed": True}, 
                              error_category=ErrorCategory.SERVER_CRASH,
                              skipped=True,
                              skip_reason="Server crashed")
        else:
            request_data = {
                "label": "Test Collection",
                "data": json.dumps({"message": "Hello from Jupyter!", "timestamp": time.time()})
            }
            
            headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
            
            response, elapsed, success, error_category, stack_trace = api.request("POST", "/data/submit", data=request_data, headers=headers)
            
            if success:
                collection_id = response.get("collection_id")
                if collection_id:
                    GLOBAL_STATE["collection_ids"].append(collection_id)
                
                print(f"\n✅ Data submitted successfully!")
                print(f"   Collection ID: {collection_id}")
                print(f"   Block Number: {response.get('block_number')}")
                print(f"   Message: {response.get('message')}")
            else:
                print(f"\n❌ Data submission failed: {response.get('error', 'Unknown error')}")
                print(f"   Error category: {error_category}")
            
            record_test_result("Submit Data (Single)", success, elapsed, response,
                              error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Submit Data (Single)", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 6. Test: Bulk Data Submission

Submit multiple collections to test throughput and performance.

In [ ]:
print_section(f"Test 4: Bulk Data Submission ({BULK_SUBMISSION_COUNT} blocks)")

try:
    # Check dependencies
    deps_met, skip_reason = check_dependencies("Create Account")
    if not deps_met:
        print(f"\n⏭️  Test skipped: {skip_reason}")
        record_test_result("Bulk Data Submission", False, 0, skipped=True, skip_reason=skip_reason)
    elif not GLOBAL_STATE["api_key"]:
        print("❌ No API key available. Create Account test did not provide an API key.")
        record_test_result("Bulk Data Submission", False, 0, skipped=True, skip_reason="No API key available")
    else:
        # Check server health
        print("🏥 Checking server health...")
        health, health_details = api.check_server_health()
        print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
        
        if health == ServerHealth.CRASHED:
            print("\n💀 Server is not responding! Cannot run bulk submission.")
            record_test_result("Bulk Data Submission", False, 0, 
                              details={"server_crashed": True}, 
                              error_category=ErrorCategory.SERVER_CRASH,
                              skipped=True,
                              skip_reason="Server crashed")
        else:
            headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
            
            start_time = time.time()
            successful_submissions = 0
            failed_submissions = 0
            submission_times = []
            error_categories_count = {}
            server_crashed = False
            
            print(f"Submitting {BULK_SUBMISSION_COUNT} collections...\n")
            
            for i in range(BULK_SUBMISSION_COUNT):
                # Check server health periodically during bulk test
                if i > 0 and i % 50 == 0:
                    health, _ = api.check_server_health()
                    if health == ServerHealth.CRASHED:
                        print(f"\n💀 Server crashed during bulk submission at iteration {i}!")
                        server_crashed = True
                        break
                
                request_data = {
                    "label": f"Bulk Test #{i+1}",
                    "data": json.dumps({
                        "iteration": i+1,
                        "timestamp": time.time(),
                        "test_type": "bulk_submission"
                    })
                }
                
                try:
                    response, elapsed, success, error_category, stack_trace = api.request("POST", "/data/submit", data=request_data, headers=headers, retry=False)
                    submission_times.append(elapsed)
                    
                    if success:
                        successful_submissions += 1
                        if response.get("collection_id"):
                            GLOBAL_STATE["collection_ids"].append(response.get("collection_id"))
                    else:
                        failed_submissions += 1
                        if error_category:
                            error_categories_count[error_category] = error_categories_count.get(error_category, 0) + 1
                        
                        # Stop if server crashed
                        if error_category == ErrorCategory.SERVER_CRASH:
                            print(f"\n💀 Server crashed at iteration {i+1}!")
                            server_crashed = True
                            break
                    
                    # Progress indicator
                    if (i + 1) % 10 == 0 or i == BULK_SUBMISSION_COUNT - 1:
                        progress = (i + 1) / BULK_SUBMISSION_COUNT * 100
                        print(f"Progress: {i+1}/{BULK_SUBMISSION_COUNT} ({progress:.1f}%) - "
                              f"Success: {successful_submissions}, Failed: {failed_submissions}")
                
                except Exception as e:
                    failed_submissions += 1
                    print(f"\n⚠️  Exception at iteration {i+1}: {str(e)}")
            
            total_elapsed = time.time() - start_time
            avg_time = sum(submission_times) / len(submission_times) if submission_times else 0
            throughput = successful_submissions / total_elapsed if total_elapsed > 0 else 0
            
            print(f"\n{'='*60}")
            print(f"📊 Bulk Submission Summary:")
            print(f"   Total attempted: {successful_submissions + failed_submissions}")
            print(f"   Successful: {successful_submissions}")
            print(f"   Failed: {failed_submissions}")
            print(f"   Success rate: {successful_submissions/(successful_submissions + failed_submissions)*100:.1f}%" if (successful_submissions + failed_submissions) > 0 else "   Success rate: N/A")
            print(f"   Total time: {total_elapsed:.2f}s")
            print(f"   Average time per submission: {avg_time:.3f}s")
            print(f"   Throughput: {throughput:.2f} submissions/second")
            if server_crashed:
                print(f"   ⚠️  SERVER CRASHED during test!")
            if error_categories_count:
                print(f"   Error categories:")
                for cat, count in error_categories_count.items():
                    print(f"      {cat}: {count}")
            print(f"{'='*60}")
            
            test_success = successful_submissions == BULK_SUBMISSION_COUNT and not server_crashed
            
            record_test_result("Bulk Data Submission", test_success, 
                              total_elapsed, {
                                  "total_attempted": successful_submissions + failed_submissions,
                                  "target": BULK_SUBMISSION_COUNT,
                                  "successful": successful_submissions,
                                  "failed": failed_submissions,
                                  "avg_time": avg_time,
                                  "throughput": throughput,
                                  "server_crashed": server_crashed,
                                  "error_categories": error_categories_count
                              },
                              error_category=ErrorCategory.SERVER_CRASH if server_crashed else None)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Bulk Data Submission", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 7. Test: List Collections

Retrieve all collections for the authenticated user.

In [ ]:
print_section("Test 5: List Collections")

try:
    # Check dependencies
    deps_met, skip_reason = check_dependencies("Create Account")
    if not deps_met:
        print(f"\n⏭️  Test skipped: {skip_reason}")
        record_test_result("List Collections", False, 0, skipped=True, skip_reason=skip_reason)
    elif not GLOBAL_STATE["api_key"]:
        print("❌ No API key available.")
        record_test_result("List Collections", False, 0, skipped=True, skip_reason="No API key available")
    else:
        # Check server health
        print("🏥 Checking server health...")
        health, health_details = api.check_server_health()
        print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
        
        if health == ServerHealth.CRASHED:
            print("\n💀 Server is not responding!")
            record_test_result("List Collections", False, 0, 
                              details={"server_crashed": True}, 
                              error_category=ErrorCategory.SERVER_CRASH,
                              skipped=True,
                              skip_reason="Server crashed")
        else:
            headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
            
            response, elapsed, success, error_category, stack_trace = api.request("GET", "/data/list", headers=headers)
            
            if success:
                collections = response.get("collections", [])
                
                print(f"\n✅ Retrieved {len(collections)} collections")
                
                if collections:
                    try:
                        # Display as DataFrame
                        df = pd.DataFrame(collections)
                        display(df.head(10))
                        
                        if len(collections) > 10:
                            print(f"\n... and {len(collections) - 10} more")
                    except Exception as e:
                        print(f"   ⚠️  Could not display as DataFrame: {str(e)}")
                        print(f"   Raw data: {collections[:3]}")  # Show first 3
            else:
                print(f"\n❌ Failed to list collections: {response.get('error', 'Unknown error')}")
                print(f"   Error category: {error_category}")
            
            record_test_result("List Collections", success, elapsed, 
                              {"count": len(collections) if success else 0},
                              error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("List Collections", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 8. Test: Decrypt Collection

Decrypt a specific collection to verify data integrity.

In [ ]:
print_section("Test 6: Decrypt Collection")

try:
    # Check dependencies
    deps_met, skip_reason = check_dependencies("Create Account", "Submit Data (Single)")
    if not deps_met:
        print(f"\n⏭️  Test skipped: {skip_reason}")
        record_test_result("Decrypt Collection", False, 0, skipped=True, skip_reason=skip_reason)
    elif not GLOBAL_STATE["api_key"]:
        print("❌ No API key available.")
        record_test_result("Decrypt Collection", False, 0, skipped=True, skip_reason="No API key available")
    elif not GLOBAL_STATE["collection_ids"]:
        print("❌ No collections available. Submit Data tests did not create any collections.")
        record_test_result("Decrypt Collection", False, 0, skipped=True, skip_reason="No collections available")
    else:
        # Check server health
        print("🏥 Checking server health...")
        health, health_details = api.check_server_health()
        print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
        
        if health == ServerHealth.CRASHED:
            print("\n💀 Server is not responding!")
            record_test_result("Decrypt Collection", False, 0, 
                              details={"server_crashed": True}, 
                              error_category=ErrorCategory.SERVER_CRASH,
                              skipped=True,
                              skip_reason="Server crashed")
        else:
            # Decrypt the first collection
            collection_id = GLOBAL_STATE["collection_ids"][0]
            headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
            
            response, elapsed, success, error_category, stack_trace = api.request("POST", f"/data/decrypt/{collection_id}", headers=headers)
            
            if success:
                print(f"\n✅ Collection decrypted successfully!")
                print(f"   Collection ID: {response.get('collection_id')}")
                print(f"   Label: {response.get('label')}")
                print(f"   Created at: {response.get('created_at')}")
                print(f"   Data: {response.get('data')}")
            else:
                print(f"\n❌ Decryption failed: {response.get('error', 'Unknown error')}")
                print(f"   Error category: {error_category}")
            
            record_test_result("Decrypt Collection", success, elapsed, response,
                              error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Decrypt Collection", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 9. Test: View Blockchain

Retrieve the entire blockchain state.

In [ ]:
print_section("Test 7: View Blockchain")

try:
    # Check server health
    print("🏥 Checking server health...")
    health, health_details = api.check_server_health()
    print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
    
    if health == ServerHealth.CRASHED:
        print("\n💀 Server is not responding!")
        record_test_result("View Blockchain", False, 0, 
                          details={"server_crashed": True}, 
                          error_category=ErrorCategory.SERVER_CRASH,
                          skipped=True,
                          skip_reason="Server crashed")
    else:
        response, elapsed, success, error_category, stack_trace = api.request("GET", "/chain")
        
        if success:
            chain_length = len(response.get("chain", []))
            node_id = response.get("node_id", "unknown")
            
            print(f"\n✅ Blockchain retrieved successfully!")
            print(f"   Node ID: {node_id}")
            print(f"   Chain length: {chain_length} blocks")
            
            if chain_length > 0:
                try:
                    latest_block = response["chain"][-1]
                    print(f"   Latest block index: {latest_block.get('index')}")
                    print(f"   Latest block timestamp: {latest_block.get('timestamp')}")
                except Exception as e:
                    print(f"   ⚠️  Could not parse latest block: {str(e)}")
        else:
            print(f"\n❌ Failed to retrieve blockchain: {response.get('error', 'Unknown error')}")
            print(f"   Error category: {error_category}")
            chain_length = 0
        
        record_test_result("View Blockchain", success, elapsed, 
                          {"chain_length": chain_length},
                          error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("View Blockchain", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 10. Test: Chain Statistics

Get analytics and statistics about the blockchain.

In [ ]:
print_section("Test 8: Chain Statistics")

try:
    # Check server health
    print("🏥 Checking server health...")
    health, health_details = api.check_server_health()
    print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
    
    if health == ServerHealth.CRASHED:
        print("\n💀 Server is not responding!")
        record_test_result("Chain Statistics", False, 0, 
                          details={"server_crashed": True}, 
                          error_category=ErrorCategory.SERVER_CRASH,
                          skipped=True,
                          skip_reason="Server crashed")
    else:
        response, elapsed, success, error_category, stack_trace = api.request("GET", "/stats")
        
        if success:
            print(f"\n✅ Statistics retrieved successfully!")
            print(f"   Total blocks: {response.get('total_blocks')}")
            print(f"   Total collections: {response.get('total_collections')}")
            print(f"   Total accounts: {response.get('total_accounts')}")
            print(f"   Avg block time: {response.get('avg_block_time_seconds', 0):.3f}s")
            
            validator_dist = response.get('validator_distribution', {})
            if validator_dist:
                print(f"\n   Validator distribution:")
                for validator, count in validator_dist.items():
                    print(f"      {validator}: {count} blocks")
        else:
            print(f"\n❌ Failed to retrieve statistics: {response.get('error', 'Unknown error')}")
            print(f"   Error category: {error_category}")
        
        record_test_result("Chain Statistics", success, elapsed, response,
                          error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Chain Statistics", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 11. Test: Node Metrics

Get performance metrics from the current node.

In [ ]:
print_section("Test 9: Node Metrics")

try:
    # Check server health
    print("🏥 Checking server health...")
    health, health_details = api.check_server_health()
    print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
    
    if health == ServerHealth.CRASHED:
        print("\n💀 Server is not responding!")
        record_test_result("Node Metrics", False, 0, 
                          details={"server_crashed": True}, 
                          error_category=ErrorCategory.SERVER_CRASH,
                          skipped=True,
                          skip_reason="Server crashed")
    else:
        response, elapsed, success, error_category, stack_trace = api.request("GET", "/metrics")
        
        if success:
            print(f"\n✅ Metrics retrieved successfully!")
            print(f"   Node ID: {response.get('node_id')}")
            print(f"   Chain length: {response.get('chain_length')}")
            print(f"   Peer count: {response.get('peer_count')}")
            print(f"   Latest block index: {response.get('latest_block_index')}")
            print(f"   Latest block timestamp: {response.get('latest_block_timestamp')}")
            print(f"   Status: {response.get('status')}")
        else:
            print(f"\n❌ Failed to retrieve metrics: {response.get('error', 'Unknown error')}")
            print(f"   Error category: {error_category}")
        
        record_test_result("Node Metrics", success, elapsed, response,
                          error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Node Metrics", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 12. Test: P2P Peers

View connected P2P peers and their reputation.

In [ ]:
print_section("Test 10: P2P Peers")

try:
    # Check server health
    print("🏥 Checking server health...")
    health, health_details = api.check_server_health()
    print(f"   Server status: {health} - {health_details.get('status', 'unknown')}")
    
    if health == ServerHealth.CRASHED:
        print("\n💀 Server is not responding!")
        record_test_result("P2P Peers", False, 0, 
                          details={"server_crashed": True}, 
                          error_category=ErrorCategory.SERVER_CRASH,
                          skipped=True,
                          skip_reason="Server crashed")
    else:
        response, elapsed, success, error_category, stack_trace = api.request("GET", "/peers")
        
        if success:
            peers = response.get('peers', [])
            reputation = response.get('reputation', {})
            
            print(f"\n✅ Peer information retrieved successfully!")
            print(f"   Connected peers: {response.get('count', 0)}")
            
            if peers:
                print(f"\n   Peer list:")
                for peer in peers:
                    rep = reputation.get(peer, 0)
                    print(f"      {peer} (reputation: {rep})")
            else:
                print(f"   No peers connected")
        else:
            print(f"\n❌ Failed to retrieve peers: {response.get('error', 'Unknown error')}")
            print(f"   Error category: {error_category}")
        
        record_test_result("P2P Peers", success, elapsed, response,
                          error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("P2P Peers", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 13. Test: Health Check

Verify node health status (used by load balancer).

In [ ]:
print_section("Test 11: Health Check")

try:
    # No dependency or health check needed since this IS the health check
    response, elapsed, success, error_category, stack_trace = api.request("GET", "/health")
    
    if success:
        print(f"\n✅ Node is healthy!")
        print(f"   Status: {response.get('status')}")
        print(f"   Node ID: {response.get('node_id')}")
        print(f"   Chain length: {response.get('chain_length')}")
        print(f"   Peer count: {response.get('peer_count')}")
        print(f"   Latest block: {response.get('latest_block')}")
    else:
        print(f"\n❌ Health check failed: {response.get('error', 'Unknown error')}")
        print(f"   Error category: {error_category}")
        print(f"   This indicates the server is either crashed, degraded, or experiencing issues")
    
    record_test_result("Health Check", success, elapsed, response,
                      error_category=error_category, stack_trace=stack_trace)

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION: {str(e)}")
    stack = traceback.format_exc()
    if VERBOSE_LOGGING:
        print(f"   Stack trace:\n{stack}")
    record_test_result("Health Check", False, 0,
                      details={"unhandled_exception": str(e)},
                      error_category=ErrorCategory.UNKNOWN,
                      stack_trace=stack)

## 14. Final Report & Visualizations

Generate comprehensive report with timing analysis and charts.

In [ ]:
print_section("Final Report & Analysis")

try:
    # Calculate summary statistics
    total_tests = len(GLOBAL_STATE["test_results"])
    successful_tests = sum(1 for r in GLOBAL_STATE["test_results"] if r["success"] and not r.get("skipped"))
    failed_tests = sum(1 for r in GLOBAL_STATE["test_results"] if not r["success"] and not r.get("skipped"))
    skipped_tests = sum(1 for r in GLOBAL_STATE["test_results"] if r.get("skipped"))
    success_rate = (successful_tests / (successful_tests + failed_tests) * 100) if (successful_tests + failed_tests) > 0 else 0
    
    total_time = sum(r["elapsed_seconds"] for r in GLOBAL_STATE["test_results"])
    avg_time = total_time / total_tests if total_tests > 0 else 0
    
    print(f"\n📊 Test Execution Summary:")
    print(f"{'='*60}")
    print(f"Total tests: {total_tests}")
    print(f"Successful: {successful_tests}")
    print(f"Failed: {failed_tests}")
    print(f"Skipped: {skipped_tests}")
    print(f"Success rate: {success_rate:.1f}% (excluding skipped)")
    print(f"Total execution time: {total_time:.2f}s")
    print(f"Average time per test: {avg_time:.3f}s")
    print(f"{'='*60}\n")
    
    # Error category analysis
    error_categories = {}
    for result in GLOBAL_STATE["test_results"]:
        if result.get("error_category"):
            cat = result["error_category"]
            error_categories[cat] = error_categories.get(cat, 0) + 1
    
    if error_categories:
        print(f"\n🔍 Error Category Breakdown:")
        print(f"{'='*60}")
        for category, count in sorted(error_categories.items(), key=lambda x: x[1], reverse=True):
            print(f"   {category}: {count} occurrences")
        print(f"{'='*60}\n")
    
    # Server health timeline
    if GLOBAL_STATE["server_health_history"]:
        print(f"\n🏥 Server Health Timeline:")
        print(f"{'='*60}")
        health_counts = {}
        for entry in GLOBAL_STATE["server_health_history"]:
            health = entry["health"]
            health_counts[health] = health_counts.get(health, 0) + 1
        
        for health, count in sorted(health_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"   {health}: {count} checks")
        
        # Check for crashes
        crashes = [e for e in GLOBAL_STATE["server_health_history"] if e["health"] == ServerHealth.CRASHED]
        if crashes:
            print(f"\n   ⚠️  SERVER CRASHES DETECTED: {len(crashes)} times")
            print(f"   First crash at: {crashes[0]['timestamp']}")
        print(f"{'='*60}\n")
    
    # Dependency chain analysis
    if GLOBAL_STATE["failed_dependencies"]:
        print(f"\n🔗 Failed Dependencies (blocking downstream tests):")
        print(f"{'='*60}")
        for dep in GLOBAL_STATE["failed_dependencies"]:
            blocked_tests = [r for r in GLOBAL_STATE["test_results"] 
                           if r.get("skip_reason") and dep in r.get("skip_reason", "")]
            print(f"   '{dep}' blocked {len(blocked_tests)} downstream test(s)")
        print(f"{'='*60}\n")
    
    # Create results DataFrame
    df_results = pd.DataFrame(GLOBAL_STATE["test_results"])
    
    print("\n📋 Detailed Results:")
    display_cols = ["test_name", "success", "skipped", "elapsed_seconds", "error_category", "server_health"]
    if all(col in df_results.columns for col in display_cols):
        display(df_results[display_cols])
    else:
        display(df_results)
    
    # Export to CSV if configured
    if EXPORT_TO_CSV:
        try:
            csv_path = f"{REPORT_OUTPUT_DIR}test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df_results.to_csv(csv_path, index=False)
            print(f"\n✅ Results exported to: {csv_path}")
        except Exception as e:
            print(f"\n⚠️  Failed to export CSV: {str(e)}")
    
    # Export to JSON if configured
    if EXPORT_TO_JSON:
        try:
            json_path = f"{REPORT_OUTPUT_DIR}test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(json_path, 'w') as f:
                json.dump({
                    "test_run_info": {
                        "timestamp": datetime.now().isoformat(),
                        "base_url": BASE_URL,
                        "bulk_submission_count": BULK_SUBMISSION_COUNT,
                        "api_timeout": API_TIMEOUT,
                        "retry_attempts": RETRY_ATTEMPTS,
                    },
                    "system_info": GLOBAL_STATE["system_info"],
                    "summary": {
                        "total_tests": total_tests,
                        "successful": successful_tests,
                        "failed": failed_tests,
                        "skipped": skipped_tests,
                        "success_rate": success_rate,
                        "total_time": total_time,
                        "avg_time": avg_time,
                        "error_categories": error_categories,
                    },
                    "server_health_summary": {
                        "total_checks": len(GLOBAL_STATE["server_health_history"]),
                        "crash_count": len([e for e in GLOBAL_STATE["server_health_history"] if e["health"] == ServerHealth.CRASHED]),
                        "timeline": GLOBAL_STATE["server_health_history"],
                    },
                    "results": GLOBAL_STATE["test_results"],
                    "global_state": {
                        "account_id": GLOBAL_STATE["account_id"],
                        "collection_count": len(GLOBAL_STATE["collection_ids"]),
                        "failed_dependencies": list(GLOBAL_STATE["failed_dependencies"]),
                    }
                }, f, indent=2)
            print(f"✅ Results exported to: {json_path}")
        except Exception as e:
            print(f"\n⚠️  Failed to export JSON: {str(e)}")
    
    # Recommendations
    print(f"\n💡 Recommendations:")
    print(f"{'='*60}")
    
    if failed_tests == 0 and skipped_tests == 0:
        print("   ✅ All tests passed! System is healthy.")
    else:
        if ErrorCategory.SERVER_CRASH in error_categories:
            print("   🚨 Server crashed during testing - check memory limits and logs")
        if ErrorCategory.TIMEOUT in error_categories:
            print("   ⏱️  Timeouts detected - consider increasing API_TIMEOUT or investigating slow endpoints")
        if ErrorCategory.CONNECTION_ERROR in error_categories:
            print("   🔌 Connection errors detected - verify network configuration and container health")
        if skipped_tests > 0:
            print(f"   ⏭️  {skipped_tests} tests skipped due to failed dependencies - fix root causes first")
        if failed_tests > successful_tests:
            print("   🔥 More failures than successes - system may be fundamentally broken")
    
    print(f"{'='*60}")

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION in reporting: {str(e)}")
    stack = traceback.format_exc()
    print(f"   Stack trace:\n{stack}")

In [ ]:
# Generate timing charts and failure analysis visualizations
try:
    if GENERATE_CHARTS and (GLOBAL_STATE["timing_data"] or GLOBAL_STATE["test_results"]):
        print("\n📊 Generating performance charts and failure analysis...\n")
        
        # Determine how many charts to create
        has_timing_data = bool(GLOBAL_STATE["timing_data"])
        has_test_data = bool(GLOBAL_STATE["test_results"])
        
        num_charts = 0
        if has_timing_data:
            num_charts += 2  # Average latency + distribution
        if has_test_data:
            num_charts += 2  # Error category pie chart + server health timeline
        
        if num_charts == 0:
            print("⚠️  No data available for charting")
        else:
            # Create figure with appropriate number of subplots
            if num_charts == 4:
                fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
                axes = [ax1, ax2, ax3, ax4]
            elif num_charts == 3:
                fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
                axes = [ax1, ax2, ax3]
            elif num_charts == 2:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
                axes = [ax1, ax2]
            else:
                fig, ax1 = plt.subplots(1, 1, figsize=(8, 6))
                axes = [ax1]
            
            chart_idx = 0
            
            # Chart 1: Average latency per endpoint
            if has_timing_data:
                endpoints = list(GLOBAL_STATE["timing_data"].keys())
                avg_times = [sum(times)/len(times) for times in GLOBAL_STATE["timing_data"].values()]
                
                axes[chart_idx].barh(endpoints, avg_times, color='steelblue')
                axes[chart_idx].set_xlabel('Average Response Time (seconds)')
                axes[chart_idx].set_title('Average Latency by Endpoint')
                axes[chart_idx].grid(axis='x', alpha=0.3)
                chart_idx += 1
                
                # Chart 2: Response time distribution
                all_times = []
                all_labels = []
                for endpoint, times in GLOBAL_STATE["timing_data"].items():
                    all_times.extend(times)
                    all_labels.extend([endpoint] * len(times))
                
                df_times = pd.DataFrame({'Endpoint': all_labels, 'Time': all_times})
                
                # Box plot for distribution
                sns.boxplot(data=df_times, y='Endpoint', x='Time', ax=axes[chart_idx])
                axes[chart_idx].set_xlabel('Response Time (seconds)')
                axes[chart_idx].set_title('Response Time Distribution')
                axes[chart_idx].grid(axis='x', alpha=0.3)
                chart_idx += 1
            
            # Chart 3: Error category pie chart
            if has_test_data:
                error_categories = {}
                for result in GLOBAL_STATE["test_results"]:
                    if result.get("error_category"):
                        cat = result["error_category"]
                        error_categories[cat] = error_categories.get(cat, 0) + 1
                
                if error_categories and chart_idx < len(axes):
                    labels = list(error_categories.keys())
                    sizes = list(error_categories.values())
                    colors = plt.cm.Set3(range(len(labels)))
                    
                    axes[chart_idx].pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
                    axes[chart_idx].set_title('Error Category Distribution')
                    chart_idx += 1
                
                # Chart 4: Server health timeline
                if GLOBAL_STATE["server_health_history"] and chart_idx < len(axes):
                    health_timeline = []
                    timestamps = []
                    
                    for i, entry in enumerate(GLOBAL_STATE["server_health_history"]):
                        health = entry["health"]
                        # Map health to numeric value for plotting
                        health_value = {
                            ServerHealth.ALIVE: 3,
                            ServerHealth.DEGRADED: 2,
                            ServerHealth.CRASHED: 1,
                            ServerHealth.UNKNOWN: 0
                        }.get(health, 0)
                        health_timeline.append(health_value)
                        timestamps.append(i)
                    
                    axes[chart_idx].plot(timestamps, health_timeline, marker='o', linewidth=2, markersize=6)
                    axes[chart_idx].set_xlabel('Health Check Number')
                    axes[chart_idx].set_ylabel('Server Health')
                    axes[chart_idx].set_title('Server Health Timeline')
                    axes[chart_idx].set_yticks([0, 1, 2, 3])
                    axes[chart_idx].set_yticklabels(['Unknown', 'Crashed', 'Degraded', 'Alive'])
                    axes[chart_idx].grid(alpha=0.3)
                    
                    # Highlight crashes
                    crash_indices = [i for i, v in enumerate(health_timeline) if v == 1]
                    if crash_indices:
                        axes[chart_idx].scatter([timestamps[i] for i in crash_indices],
                                               [health_timeline[i] for i in crash_indices],
                                               color='red', s=200, alpha=0.5, zorder=5,
                                               label='Crashes')
                        axes[chart_idx].legend()
            
            plt.tight_layout()
            
            # Save chart
            try:
                chart_path = f"{REPORT_OUTPUT_DIR}performance_chart_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
                plt.savefig(chart_path, dpi=150, bbox_inches='tight')
                print(f"✅ Chart saved to: {chart_path}")
            except Exception as e:
                print(f"⚠️  Failed to save chart: {str(e)}")
            
            plt.show()
    
    print("\n✅ Report generation complete!")
    print(f"\n{'='*60}")
    print("📁 Artifacts Generated:")
    print(f"   - Test results DataFrame (displayed above)")
    if EXPORT_TO_CSV:
        print(f"   - CSV export: {REPORT_OUTPUT_DIR}test_results_*.csv")
    if EXPORT_TO_JSON:
        print(f"   - JSON export: {REPORT_OUTPUT_DIR}test_results_*.json")
    if GENERATE_CHARTS:
        print(f"   - Performance charts: {REPORT_OUTPUT_DIR}performance_chart_*.png")
    print(f"{'='*60}")

except Exception as e:
    print(f"\n💥 UNHANDLED EXCEPTION in chart generation: {str(e)}")
    stack = traceback.format_exc()
    print(f"   Stack trace:\n{stack}")

## Summary

✅ All tests completed with comprehensive error handling and resilience!

**Enhanced Features:**
- 🛡️ **Error-Resistant Design**: Every test wrapped in try-except blocks to prevent cascading failures
- 🏥 **Server Health Monitoring**: Automatic health checks before each test to detect server crashes
- 🔄 **Automatic Retry Logic**: Failed requests retry with exponential backoff (configurable via `RETRY_ATTEMPTS`)
- 🔗 **Dependency Tracking**: Tests automatically skip if prerequisites failed, preventing false positives
- 📊 **Comprehensive Diagnostics**: Error categorization, server health timeline, dependency chain analysis
- 💻 **System Information**: Captures CPU, memory, disk, platform info dynamically
- 📈 **Enhanced Reporting**: Failure analysis with actionable recommendations

**Error Categories Tracked:**
- `connection_error` - Cannot connect to server
- `timeout` - Request exceeded timeout limit
- `http_error` - HTTP error status (4xx, 5xx)
- `server_crash` - Server completely unresponsive
- `json_parse_error` - Invalid JSON response
- `authentication_error` - Auth failures (401, 403)
- `unknown` - Unhandled exception types

**Artifacts Generated:**
- 📋 Test Results DataFrame (displayed in notebook)
- 📄 CSV Export: `{REPORT_OUTPUT_DIR}test_results_*.csv`
- 📦 JSON Export: `{REPORT_OUTPUT_DIR}test_results_*.json` (includes system info, server health timeline, full test metadata)
- 📊 Performance Charts: `{REPORT_OUTPUT_DIR}performance_chart_*.png` (latency, distribution, error breakdown, health timeline)

**Global State Variables:**
- `GLOBAL_STATE["api_key"]` - API key for authentication
- `GLOBAL_STATE["account_id"]` - Account ID
- `GLOBAL_STATE["collection_ids"]` - List of created collection IDs
- `GLOBAL_STATE["test_results"]` - Detailed results for all tests
- `GLOBAL_STATE["system_info"]` - System/environment information
- `GLOBAL_STATE["server_health_history"]` - Timeline of server health checks
- `GLOBAL_STATE["failed_dependencies"]` - Set of tests that failed (blocking downstream tests)

**Next Steps:**
1. Review the detailed results and recommendations above
2. Check exported JSON for full test metadata and system info
3. Analyze performance charts to identify bottlenecks
4. If server crashed, check container logs: `docker logs <container_name>`
5. Adjust `BULK_SUBMISSION_COUNT` or `API_TIMEOUT` based on results